Data Extraction for Blackcoffer Assignment

In [2]:
#Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Importing packages

import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Importing input file
df=pd.read_csv('/content/Input (1).csv')[['URL_ID','URL']]

In [6]:
len(df)

114

In [7]:
df

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [8]:
#Looping through each row in the dataframe
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # Making a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #Creating a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #Finding title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue

   #Finding text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  #Writing the obtained Title and Text to the file
  file_name = '/content/drive/MyDrive/mywork/title_text.txt' + str(url_id) + '.txt'
  with open(file_name, 'w') as file:
    file.write(title + '\n' + article)


can't get title of 11668.0
can't get title of 17671.4


In [9]:
# Directories
text_dir = "/content/drive/MyDrive/mywork/dataextraction/title_text"
stopwords_dir = "/content/drive/MyDrive/mywork/StopWords"
sentiment_dir = "/content/drive/MyDrive/mywork/MasterDictionary"

#Loading and storing all the Stop Words from the stopwords directory to the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

#Loading all the text files from the directory and storing in a list
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r') as f:
    text = f.read()
#Tokenizing the given text file
    words = word_tokenize(text)
#Removing the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
#Adding each filtered token of each file into a list
    docs.append(filtered_text)

#Storing positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentiment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentiment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentiment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

#Collect and calculating the positive and negative words score from each file
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#Iterating through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [10]:
# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

import os
import nltk
from nltk.corpus import stopwords

avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

stopwords_set = set(stopwords.words('english'))

def measure(file):
  with open(os.path.join(text_dir, file),'r') as f:
    text = f.read()
# Removing punctuations
    text = re.sub(r'[^\w\s.]','',text)
# Spliting into sentences
    sentences = text.split('.')
# Total number of sentences in a file
    num_sentences = len(sentences)
# Total words in the file
    words = [word  for word in text.split() if word.lower() not in stopwords ]
    num_words = len(words)

# Complex words are words in the text that contain more than two syllables.
    complex_words = []
    for word in words:
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
    syllable_count = 0
    syllable_words =[]
    for word in words:
      if word.endswith('es'):
        word = word[:-2]
      elif word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
      if syllable_count_word >= 1:
        syllable_words.append(word)
        syllable_count += syllable_count_word

      try:
          avg_sentence_length = num_words / num_sentences
      except ZeroDivisionError:
          avg_sentence_len = 0  # Handle the case where num_sentences is zero

      try:
          avg_syllable_word_count = syllable_count / len(syllable_words)
      except ZeroDivisionError:
          avg_syllable_word_count = 0  # Handle the case where len(syllable_words) is zero

      try:
          Percent_Complex_words = len(complex_words) / num_words
      except ZeroDivisionError:
          Percent_Complex_words = 0  # Handle the case where num_words is zero

      Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)




      return avg_sentence_length, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

  # iterate through each file or doc
for file in os.listdir(text_dir):
    result = measure(file)
    if result is not None:
        x, y, z, a, b = result
        avg_sentence_length.append(x)
        Percentage_of_Complex_words.append(y)
        Fog_Index.append(z)
        complex_word_count.append(a)
        avg_syllable_word_count.append(b)

    else:
        print(f"Measurement couldn't be performed for file: {file}")

In [11]:
# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by
# Removing the stop words (using stopwords class of nltk package).
# Removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)

# To calculate Personal Pronouns mentioned in the text, we use regex to find
# The counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name "US" is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [19]:
from google.colab import files

output_df = pd.read_excel('/content/Output Data Structure (2).xlsx')

output_df.drop([44-37,57-37,144-37], axis = 0, inplace=True)

variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# Initializing the DataFrame with the appropriate number of columns
output_df = pd.DataFrame(columns=['positive_score', 'negative_score', 'polarity_score', 'subjectivity_score',
                                  'avg_sentence_length','Percentage_of_Complex_words','Fog_Index','avg_sentence_length','complex_word_count',
                                  'word_count','avg_syllable_word_count','pp_count','average_word_length'])

# Assigning the values from variables to specific columns
for i, var in enumerate(variables):
    column_name = output_df.columns[i]
    output_df[column_name] = var

output_df.to_excel('Output_Data.xlsx')
